## Record on Proactive Label Relaxation on Fer2013 DATASET 







In [1]:
## Loading dataset from .npy files

import numpy as np
from keras.utils import to_categorical

X_train = np.load('X_train_Fer2013.npy')
#X_train_2 = np.load('evae_conditional_fer2013_sample_neutral_sample_neutral_a=0_v1.npy')
#X_train_2 = X_train_2.reshape((X_train_2.shape[0], -1))
X_test = np.load('X_test_Fer2013.npy')
#y_train = np.load('y_train_Fer2013.npy')
y_test = np.load('y_test_Fer2013.npy')

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
### Loading crafted soft training labels and tesitng labels 

import keras.backend as K

y_test  = to_categorical(y_test)
#y_train = to_categorical(y_train)
y_train = np.load('proactive_label_fer2013_v7_divide12_bound3.0*6.npy')


In [3]:
y_train[2]

array([0.0429794 , 0.04216032, 0.74543467, 0.04297165, 0.04265694,
       0.04188894, 0.04190807])

In [4]:
## convert image size for CNN 
img_rows, img_cols = 48, 48

if K.image_data_format() == 'channels_first':
    X_train= X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [5]:
from keras.layers import Conv2D, BatchNormalization, AveragePooling2D, PReLU, Dropout, Dense
from keras.layers import Flatten, Activation
from keras.models import Sequential

num_classes = 7

model_pr = Sequential()

model_pr.add(Conv2D(16, (7, 7), padding='same',
                        input_shape=input_shape, name = 'block1_conv1'))
model_pr.add(PReLU())
model_pr.add(BatchNormalization())
model_pr.add(AveragePooling2D(pool_size=(5, 5),strides=(2, 2), padding='same', name = 'block1_pool1'))
model_pr.add(Dropout(.5))

model_pr.add(Conv2D(32, (5, 5), padding='same', name = 'block2_conv1'))
model_pr.add(PReLU())
model_pr.add(BatchNormalization())
model_pr.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same', name = 'block2_pool1'))
model_pr.add(Dropout(.5))

model_pr.add(Conv2D(32, (3, 3), padding='same', name = 'block3_conv1'))
model_pr.add(PReLU())
model_pr.add(BatchNormalization())
model_pr.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same', name = 'block3_pool1'))
model_pr.add(Dropout(.5))

model_pr.add(Flatten())
model_pr.add(Dense(512, name = 'fc1'))
model_pr.add(PReLU())
model_pr.add(Dropout(0.5))
model_pr.add(Dense(num_classes))
model_pr.add(Activation('softmax', name ='predictions'))


from keras.losses import categorical_crossentropy
from keras.optimizers import Adam



#sgd_wn = SGDWithWeightnorm(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_pr.compile(loss = categorical_crossentropy, optimizer = Adam(), metrics = ['accuracy'])

# callback history 
from keras.callbacks import EarlyStopping,TensorBoard,History, ReduceLROnPlateau

#earlyStop  = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 0, verbose = 0, mode = 'auto')

#tbcallback = TensorBoard(log_dir = './smooth_logs', histogram_freq = 0, write_graph = True, write_images = True)

hist_vanilla    = History()

rdLR       = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10, verbose = 0, 
                              mode = 'auto', min_delta = 0.001, cooldown = 0, min_lr = 0)

model_vanilla_callbacks = [hist_vanilla, rdLR]

In [6]:
num_epochs = 150 #cease to improve after 50 epoches 
batch_size = 100

model_pr.fit(X_train,y_train, 
          batch_size = batch_size,
          epochs = num_epochs,
          verbose= 2,
          callbacks=model_vanilla_callbacks,
          shuffle=True,
          validation_data =(X_test,y_test)
          )

Train on 32298 samples, validate on 3589 samples
Epoch 1/150
 - 188s - loss: 1.8566 - acc: 0.2875 - val_loss: 1.7303 - val_acc: 0.3168
Epoch 2/150
 - 188s - loss: 1.7311 - acc: 0.3490 - val_loss: 1.7078 - val_acc: 0.3536
Epoch 3/150
 - 189s - loss: 1.6882 - acc: 0.3841 - val_loss: 1.5916 - val_acc: 0.3812
Epoch 4/150
 - 189s - loss: 1.6566 - acc: 0.4012 - val_loss: 1.4843 - val_acc: 0.4366
Epoch 5/150
 - 189s - loss: 1.6301 - acc: 0.4175 - val_loss: 1.4840 - val_acc: 0.4372
Epoch 6/150
 - 190s - loss: 1.6092 - acc: 0.4305 - val_loss: 1.4496 - val_acc: 0.4505
Epoch 7/150
 - 190s - loss: 1.5872 - acc: 0.4431 - val_loss: 1.3833 - val_acc: 0.4765
Epoch 8/150
 - 190s - loss: 1.5682 - acc: 0.4581 - val_loss: 1.4420 - val_acc: 0.4712
Epoch 9/150
 - 190s - loss: 1.5503 - acc: 0.4705 - val_loss: 1.3799 - val_acc: 0.4979
Epoch 10/150
 - 191s - loss: 1.5376 - acc: 0.4750 - val_loss: 1.3422 - val_acc: 0.4909
Epoch 11/150
 - 190s - loss: 1.5197 - acc: 0.4857 - val_loss: 1.3120 - val_acc: 0.5057
Epo

KeyboardInterrupt: 

In [14]:
import pandas as pd

training_history = hist_vanilla.history

#pd.DataFrame(training_history).to_csv('hist_bound_2.5_x7_150epoch.csv')

In [45]:
from keras.layers import Conv2D, BatchNormalization, AveragePooling2D, PReLU, Dropout, Dense
from keras.layers import Flatten, Activation,Subtract
from keras.models import Sequential
from keras.layers import MaxPooling2D, ReLU
from keras.layers import Concatenate
from keras import Model
from keras.layers import Dot,Multiply,Subtract,Add,BatchNormalization, Average, Maximum
from keras.layers import Lambda, Input, Dense
from keras import backend as K

input_shape = (48,48,1)

num_classes = 7

x = Input(shape=input_shape, name='input')
###  Tryout more variations of initlisation techniques 
### without any pre-training 
### without any weight initilaistion 

##
h1 = Conv2D(16, (7, 7), padding='same',
                        input_shape=input_shape, name = 'block1_conv1', activation = 'relu')(x)

h1 = BatchNormalization()(h1)
h1 = AveragePooling2D(pool_size=(5, 5),strides=(2, 2), padding='same')(h1)
h1 = Dropout(.5)(h1)

##
h2 = Conv2D(32, (5, 5), padding='same',
                        input_shape=input_shape, name = 'block1_conv2', activation = 'relu')(h1)
h2 = BatchNormalization()(h2)
h2 = AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same')(h2)
h2 = Dropout(.5)(h2)

##
h3 = Conv2D(32, (3, 3), padding='same',
                        input_shape=input_shape, name = 'block1_conv3', activation = 'relu')(h2)
h3 = BatchNormalization()(h3)
h3 = AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same')(h3)
h3 = Dropout(.5)(h3)


h3 = Flatten()(h3)
h4 = Dense(512, name = 'fc1', activation = 'relu')(h3)
h5 = Dropout(0.5)(h4)
h5 = BatchNormalization()(h5)
h_class = Dense(50, activation = 'relu')(h5)
h_class = Dense(7, activation='softmax')(h_class)
#h_final.add(Dense(num_classes))
#model_p1.add(Activation('softmax', name ='predictions'))

'''
input_shape_2 = (48*48,)
x_2 = Input(shape = input_shape_2)

h_2 = Dense(500, activation='relu')(x_2)

h_2 = Dense(200,activation='relu')(h_2)

h_2 = Dense(50,activation='relu')(h_2)

h_2 = BatchNormalization()(h_2)

h_final = Subtract()([h_class, h_2])

h_final_2 = Add()([h_final, h_class])
h_class = Dense(7, activation='softmax')(h_final_2)
'''
model_p1 = Model(x, h_class)



In [46]:
"""
## Aux network for 
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras import backend as K

x = Input(shape = X_train_2.shape)

x = Dense(500, activation='relu')(x)

x = Dense(200,activation='relu')(x)

x = BatchNormalization()(x)

x = Dense(7,activaition = 'prelu')(x)

"""

"\n## Aux network for \nfrom keras.layers import Lambda, Input, Dense\nfrom keras.models import Model\nfrom keras import backend as K\n\nx = Input(shape = X_train_2.shape)\n\nx = Dense(500, activation='relu')(x)\n\nx = Dense(200,activation='relu')(x)\n\nx = BatchNormalization()(x)\n\nx = Dense(7,activaition = 'prelu')(x)\n\n"

In [47]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam



#sgd_wn = SGDWithWeightnorm(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_p1.compile(loss = categorical_crossentropy, optimizer = Adam(), metrics = ['accuracy'])

# callback history 
from keras.callbacks import EarlyStopping,TensorBoard,History, ReduceLROnPlateau

#earlyStop  = EarlyStopping(monitor = 'val_loss',  patience = 10)

#tbcallback = TensorBoard(log_dir = './smooth_logs', histogram_freq = 0, write_graph = True, write_images = True)

hist_vanilla = History()

#rdLR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10, verbose = 0, 
                              #mode = 'auto', cooldown = 0, min_lr = 0)

model_callbacks = [hist_vanilla]

In [48]:
num_epochs = 150 # 
batch_size = 100

model_p1.fit([X_train,X_train_2],y_train, 
          batch_size = batch_size,
          epochs = num_epochs,
          verbose= 2,
          callbacks=model_callbacks,
          shuffle=True,
          )

Epoch 1/150
 - 212s - loss: 2.1082 - acc: 0.2388
Epoch 2/150
 - 188s - loss: 1.7714 - acc: 0.2937
Epoch 3/150
 - 185s - loss: 1.6808 - acc: 0.3358
Epoch 4/150
 - 187s - loss: 1.6114 - acc: 0.3708
Epoch 5/150
 - 187s - loss: 1.5543 - acc: 0.3932
Epoch 6/150
 - 185s - loss: 1.5066 - acc: 0.4163
Epoch 7/150
 - 192s - loss: 1.4732 - acc: 0.4323
Epoch 8/150
 - 189s - loss: 1.4349 - acc: 0.4461
Epoch 9/150
 - 189s - loss: 1.4134 - acc: 0.4515
Epoch 10/150
 - 185s - loss: 1.3856 - acc: 0.4677
Epoch 11/150
 - 206s - loss: 1.3745 - acc: 0.4709
Epoch 12/150
 - 186s - loss: 1.3553 - acc: 0.4772
Epoch 13/150
 - 183s - loss: 1.3339 - acc: 0.4856
Epoch 14/150
 - 184s - loss: 1.3257 - acc: 0.4881
Epoch 15/150
 - 189s - loss: 1.3102 - acc: 0.4964
Epoch 16/150


KeyboardInterrupt: 